# Micro:bit project
Project description
The company "Strawbees" for which this project was carried out sells STEAM learning materials that are compatible with the micro:bit device.

The comlany collabs with Micro:bit Educational Foundation, so is able to get a data from them about their purchases as part of the exchange.
Strawbees uses a CRM system (Hubspot) to keep track of US school districts, schools, educational resellers, e.t.c. they maintain a relationship with.
A Micro:bit dataset contains a list of US schools districts that had spent part of their budget on acquiring micro:bit devices. These schools districts are of high interest for establishing communications for Strawbees. However, in order to identify these districts within the company's CRM, the records from the Microbit dataset need to be linked to the CRM records. 
The project involves matching these two datasets through exact and fuzzy linking techniques.

# Skills developed:
Importing data into a script for processing

Data cleaning

Data manipulation

Joining datasets

Record linkage (fuzzy matching)

In the interests of the company, data on the total amounts and number of purchases in the micro:bit dataset are hidden in accordance with the NDA.

# Micro:bit project outline:

CSV_files for the project:
1. Micro:bit school districts file from collaborator - number of district to match - 1897.
2. US school districts file from NCES - official source of National Center for Education Statistics 
https://nces.ed.gov/ccd/files.asp#Fiscal:2,LevelId:5,SchoolYearId:38,Page:1 - file location.
Total number of US school districts - 19627.

Part 1: Preparing micro:bit district file for the import.

Part 2: Preparing US school districts import file.

Part 3: Matching through exact name.

Part 4: Record linkage through similar names. 

Part 5: Merging resulting dataframes for the final version.

# Imports required for the project
Importing Pandas library for data manipulation and Fuzz (a string matching library) for fuzzy string matching operations.
Fuzzy string matching refers to finding strings that are approximately rather than exactly equal, allowing for minor differences like typos or variations.

In [1]:
import pandas as pd
from thefuzz import fuzz, process

# Exploring and cleaning data

In [2]:
data_nces = pd.read_csv("school_districts_NCES.csv",dtype={'LEAID':str})

C:\Users\zalis\AppData\Local\Temp\ipykernel_15152\3209038505.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data_nces = pd.read_csv("school_districts_NCES.csv",dtype={'LEAID':str})


In [3]:
data_microbit = pd.read_csv("school_districts_micro_bit.csv")

In [4]:
data_nces.head(2)

SCHOOL_YEAR  FIPST STATENAME  ST                LEA_NAME  STATE_AGENCY_NO  \
0   2023-2024      1   ALABAMA  AL  Alabama Youth Services                1   
1   2023-2024      1   ALABAMA  AL        Albertville City                1   

   UNION ST_LEAID    LEAID                     MSTREET1  ... G_11_OFFERED  \
0    NaN   AL-210  0100002  1000 Industrial School Road  ...          Yes   
1    NaN   AL-101  0100005          8379 US Highway 431  ...          Yes   

  G_12_OFFERED G_13_OFFERED G_UG_OFFERED  G_AE_OFFERED  GSLO GSHI  LEVEL  \
0          Yes           No           No            No    KG   12  Other   
1          Yes           No           No            No    PK   12  Other   

     IGOFFERED OPERATIONAL_SCHOOLS  
0  As reported                   0  
1  As reported                   6  

[2 rows x 58 columns]

In [5]:
data_microbit.head(2)

Agency Unnamed: 1  \
0  State College Area School District, Pennsylvania        NaN   
1               Carson City School District, Nevada        NaN   

  Total spend (approx) Number of purchases  
0                  NDA                 NDA  
1                  NDA                 NDA

In [41]:
#Number of districts to match
data_microbit.shape[0]

1896

Part 1. Preparing micro:bit district file for the import
Exploring Row partner's data requirued for the project, filter out unneeded, empty data. 
(According NDA data in columns 'Total spend (approx)' and 'Number of purchases' are  hidden)
Spliting 'Agency' column into 'District Name' and 'State'

In [7]:
data_microbit.drop(labels=["Unnamed: 1", "Number of purchases"], axis=1, inplace=True)
data_microbit.head(2)

Agency Total spend (approx)
0  State College Area School District, Pennsylvania                  NDA
1               Carson City School District, Nevada                  NDA

In [8]:
data_microbit[['district','state']] = data_microbit["Agency"].str.split(pat=", ", n=1, expand=True)
data_microbit.head(2)

Agency Total spend (approx)  \
0  State College Area School District, Pennsylvania                  NDA   
1               Carson City School District, Nevada                  NDA   

                             district         state  
0  State College Area School District  Pennsylvania  
1         Carson City School District        Nevada

Part 2: Preparing US school districts import file.

Filtering out unneeded columns (keep at least STATENAME, LEAID, LEA_NAME)
Important: Importing LEAID as string rather than integer. Using dtype parameter from pandas.read_csv (using this source for the project and skills developing).
Adding prefix (NCES_District-) to LEAID in order to create National Data ID column (similar with the Property name in Hubspot SRM Strawbees DataBase).
Capitalising DISTRICTNAME, STATENAME columns (other if needed) to match the values in micro.bit file.
Capitalising all distcricts names in both files for more accurate matches.

In [9]:
data_nces.columns

Index(['SCHOOL_YEAR', 'FIPST', 'STATENAME', 'ST', 'LEA_NAME',
       'STATE_AGENCY_NO', 'UNION', 'ST_LEAID', 'LEAID', 'MSTREET1', 'MSTREET2',
       'MSTREET3', 'MCITY', 'MSTATE', 'MZIP', 'MZIP4', 'LSTREET1', 'LSTREET2',
       'LSTREET3', 'LCITY', 'LSTATE', 'LZIP', 'LZIP4', 'PHONE', 'WEBSITE',
       'SY_STATUS', 'SY_STATUS_TEXT', 'UPDATED_STATUS', 'UPDATED_STATUS_TEXT',
       'EFFECTIVE_DATE', 'LEA_TYPE', 'LEA_TYPE_TEXT', 'OUT_OF_STATE_FLAG',
       'CHARTER_LEA', 'CHARTER_LEA_TEXT', 'NOGRADES', 'G_PK_OFFERED',
       'G_KG_OFFERED', 'G_1_OFFERED', 'G_2_OFFERED', 'G_3_OFFERED',
       'G_4_OFFERED', 'G_5_OFFERED', 'G_6_OFFERED', 'G_7_OFFERED',
       'G_8_OFFERED', 'G_9_OFFERED', 'G_10_OFFERED', 'G_11_OFFERED',
       'G_12_OFFERED', 'G_13_OFFERED', 'G_UG_OFFERED', 'G_AE_OFFERED', 'GSLO',
       'GSHI', 'LEVEL', 'IGOFFERED', 'OPERATIONAL_SCHOOLS'],
      dtype='object')

In [10]:
labels_to_drop = ["SCHOOL_YEAR", "FIPST", "ST", "STATE_AGENCY_NO", "UNION", "ST_LEAID", "MSTREET2", "MSTREET3", "MSTATE", "MZIP", "MZIP4", "LSTREET1", "LSTREET2", "LSTREET3", 
"LCITY", "LSTATE", "LZIP", "LZIP4", "PHONE", "SY_STATUS", "SY_STATUS_TEXT", "UPDATED_STATUS", "UPDATED_STATUS_TEXT",
"EFFECTIVE_DATE", "LEA_TYPE", "LEA_TYPE_TEXT", "OUT_OF_STATE_FLAG", "CHARTER_LEA", "CHARTER_LEA_TEXT", "NOGRADES", "G_PK_OFFERED",
"G_KG_OFFERED", "G_1_OFFERED", "G_2_OFFERED", "G_3_OFFERED", "G_4_OFFERED", "G_5_OFFERED", "G_6_OFFERED", "G_7_OFFERED",
"G_8_OFFERED", "G_9_OFFERED", "G_10_OFFERED", "G_11_OFFERED", "G_12_OFFERED", "G_13_OFFERED", "G_UG_OFFERED", "G_AE_OFFERED", "GSLO",
"GSHI", "LEVEL", "IGOFFERED", "OPERATIONAL_SCHOOLS"]

In [11]:
data_nces.drop(labels=labels_to_drop, axis=1, inplace=True)
data_nces.head(2)

STATENAME                LEA_NAME    LEAID                     MSTREET1  \
0   ALABAMA  Alabama Youth Services  0100002  1000 Industrial School Road   
1   ALABAMA        Albertville City  0100005          8379 US Highway 431   

         MCITY                                          WEBSITE  
0     Mt Meigs  http://www.dys.alabama.gov/school-district.html  
1  Albertville                         http://www.albertk12.org

In [12]:
prefix = 'NCES_District-'
data_nces['LEAID'] = prefix + data_nces['LEAID']
data_nces.head(2)

STATENAME                LEA_NAME                  LEAID  \
0   ALABAMA  Alabama Youth Services  NCES_District-0100002   
1   ALABAMA        Albertville City  NCES_District-0100005   

                      MSTREET1        MCITY  \
0  1000 Industrial School Road     Mt Meigs   
1          8379 US Highway 431  Albertville   

                                           WEBSITE  
0  http://www.dys.alabama.gov/school-district.html  
1                         http://www.albertk12.org

In [13]:
mapping = {'LEAID': 'NATIONAL DATA ID', 'LEA_NAME': 'DISTRICT_NAME', 'MSTREET1': 'STREET_ADDRESS', 'MCITY': 'CITY', 
'WEBSITE': 'COMPANY_DOMAIN'}
data_nces = data_nces.rename(columns=mapping)
data_nces.head(2)

STATENAME           DISTRICT_NAME       NATIONAL DATA ID  \
0   ALABAMA  Alabama Youth Services  NCES_District-0100002   
1   ALABAMA        Albertville City  NCES_District-0100005   

                STREET_ADDRESS         CITY  \
0  1000 Industrial School Road     Mt Meigs   
1          8379 US Highway 431  Albertville   

                                    COMPANY_DOMAIN  
0  http://www.dys.alabama.gov/school-district.html  
1                         http://www.albertk12.org

In [14]:
total_districts = len(data_nces)
print(f"Number of total districts in the dataframe: {total_districts}")

Number of total districts in the dataframe: 19627


In [15]:
data_nces['DISTRICT_NAME'] = data_nces['DISTRICT_NAME'].str.upper()
data_nces.head(3)

STATENAME           DISTRICT_NAME       NATIONAL DATA ID  \
0   ALABAMA  ALABAMA YOUTH SERVICES  NCES_District-0100002   
1   ALABAMA        ALBERTVILLE CITY  NCES_District-0100005   
2   ALABAMA         MARSHALL COUNTY  NCES_District-0100006   

                STREET_ADDRESS          CITY  \
0  1000 Industrial School Road      Mt Meigs   
1          8379 US Highway 431   Albertville   
2       12380 US Highway 431 S  Guntersville   

                                    COMPANY_DOMAIN  
0  http://www.dys.alabama.gov/school-district.html  
1                         http://www.albertk12.org  
2                       http://www.marshallk12.org

# Joining data_microbit with data_nces dataframes.

Part 3: Matching through exact name.

Renaming columns in the right order.
Merging/joining micro:bit dataframe with NCES dataframe through exact District name and State name 
(Merge type (how) affects the results - I use 'Left' in order to keep all rows from the left dataframe (data_microbit), and only matching rows from the right (data_nces).)
Output: micro:bit dataframe has a column with National Data ID, which is empty for some districts (those without exact match) and filled for those with exact.
Spliting resulting dataframe into those rows with National data ID - to be imported; and the ones without National data ID - be able to work on Part 4 of the project.

In [16]:
data_microbit = data_microbit.rename(columns={'district': 'DISTRICT_NAME', 'state': 'STATENAME'})
data_microbit.head(3)

Agency Total spend (approx)  \
0  State College Area School District, Pennsylvania                  NDA   
1               Carson City School District, Nevada                  NDA   
2           Gwinnett County Public Schools, Georgia                  NDA   

                        DISTRICT_NAME     STATENAME  
0  State College Area School District  Pennsylvania  
1         Carson City School District        Nevada  
2      Gwinnett County Public Schools       Georgia

In [17]:
data_microbit['STATENAME'] = data_microbit['STATENAME'].str.upper()
data_microbit.head(3)

Agency Total spend (approx)  \
0  State College Area School District, Pennsylvania                  NDA   
1               Carson City School District, Nevada                  NDA   
2           Gwinnett County Public Schools, Georgia                  NDA   

                        DISTRICT_NAME     STATENAME  
0  State College Area School District  PENNSYLVANIA  
1         Carson City School District        NEVADA  
2      Gwinnett County Public Schools       GEORGIA

In [18]:
data_microbit['DISTRICT_NAME'] = data_microbit['DISTRICT_NAME'].str.upper()
data_microbit.head(3)

Agency Total spend (approx)  \
0  State College Area School District, Pennsylvania                  NDA   
1               Carson City School District, Nevada                  NDA   
2           Gwinnett County Public Schools, Georgia                  NDA   

                        DISTRICT_NAME     STATENAME  
0  STATE COLLEGE AREA SCHOOL DISTRICT  PENNSYLVANIA  
1         CARSON CITY SCHOOL DISTRICT        NEVADA  
2      GWINNETT COUNTY PUBLIC SCHOOLS       GEORGIA

In [19]:
data_microbit.dtypes

Agency                  object
Total spend (approx)    object
DISTRICT_NAME           object
STATENAME               object
dtype: object

In [20]:
#merging dataframes
merged_data_microbit_nces = pd.merge(data_microbit, data_nces, 
                     left_on=['DISTRICT_NAME', 'STATENAME'], 
                     right_on=['DISTRICT_NAME', 'STATENAME'],
                     how='left') 

merged_data_microbit_nces.head(10)

Agency Total spend (approx)  \
0   State College Area School District, Pennsylvania                  NDA   
1                Carson City School District, Nevada                  NDA   
2            Gwinnett County Public Schools, Georgia                  NDA   
3          Tracy Unified School District, California                  NDA   
4                      Pasco County Schools, Florida                  NDA   
5           Georgia Institute of Technology, Georgia                  NDA   
6                      Harmony Public Schools, Texas                  NDA   
7                   Modesto City Schools, California                  NDA   
8  Stanislaus County Office of Education, California                  NDA   
9       Hawaii State Department of Education, Hawaii                  NDA   

                           DISTRICT_NAME     STATENAME       NATIONAL DATA ID  \
0     STATE COLLEGE AREA SCHOOL DISTRICT  PENNSYLVANIA                    NaN   
1            CARSON CITY SCHOOL DISTRICT        NEVADA                    NaN   
2         GWINNETT COUNTY PUBLIC SCHOOLS       GEORGIA                    NaN   
3          TRACY UNIFIED SCHOOL DISTRICT    CALIFORNIA                    NaN   
4                   PASCO COUNTY SCHOOLS       FLORIDA                    NaN   
5        GEORGIA INSTITUTE OF TECHNOLOGY       GEORGIA                    NaN   
6                 HARMONY PUBLIC SCHOOLS         TEXAS                    NaN   
7                   MODESTO CITY SCHOOLS    CALIFORNIA  NCES_District-0601330   
8  STANISLAUS COUNTY OFFICE OF EDUCATION    CALIFORNIA  NCES_District-0691041   
9   HAWAII STATE DEPARTMENT OF EDUCATION        HAWAII                    NaN   

   STREET_ADDRESS     CITY              COMPANY_DOMAIN  
0             NaN      NaN                         NaN  
1             NaN      NaN                         NaN  
2             NaN      NaN                         NaN  
3             NaN      NaN                         NaN  
4             NaN      NaN                         NaN  
5             NaN      NaN                         NaN  
6             NaN      NaN                         NaN  
7  426 Locust St.  Modesto  http://www.monet.k12.ca.us  
8      1100 H St.  Modesto      http://www.stancoe.org  
9             NaN      NaN                         NaN

In [42]:
total_districts = len(merged_data_microbit_nces)
print(f"Number of total districts in the merged dataframe: {total_districts}")

Number of total districts in the merged dataframe: 1897


In [22]:
#sampling dataframes with matched and non-matched school districts
nationaldataid_present = merged_data_microbit_nces[merged_data_microbit_nces['NATIONAL DATA ID'].notna()]
nationaldataid_missing = merged_data_microbit_nces[merged_data_microbit_nces['NATIONAL DATA ID'].isna()]

In [23]:
nationaldataid_present.shape[0]

418

In [24]:
nationaldataid_missing.shape[0]

1479

# Record linkage / fuzzy matching

For fuzzy matching - Skills based on course "Cleaning data with pandas" in DataCamp.

In [25]:
nationaldataid_present.to_csv('nationaldataid_present.csv', index=False)
nationaldataid_missing.to_csv('nationaldataid_missing.csv', index=False)

In [26]:
# Fuzzy matching code

# Initialize needed lists to store results
match_count = 0
matched_ids = []
street = []
city = []
domain = []
names = []

# Iterate through each row in nationaldataid_missing
for index, row in nationaldataid_missing.iterrows():
    district_to_match = row['DISTRICT_NAME']
    state = row['STATENAME']
    
    # Convert values to strings 
    district_to_match = str(district_to_match) if not pd.isna(district_to_match) else 'Unknown'
    state = str(state) if not pd.isna(state) else 'Unknown'
    
    #print(f"Matching for: {district_to_match}: {state}")

    data_nces['Match_Score'] = data_nces['DISTRICT_NAME'].apply(lambda x: fuzz.WRatio(district_to_match, str(x)))
    
    # Filter rows by state and sort by match score
    best_match_row = data_nces[(data_nces['STATENAME'] == state)].sort_values('Match_Score', ascending=False).reset_index()
    
    # How we check if we have a matching row
    if not best_match_row.empty:
        best_match_name = best_match_row.loc[0, 'DISTRICT_NAME']
        national_id = best_match_row.loc[0, 'NATIONAL DATA ID']
        match_street = best_match_row.loc[0, 'STREET_ADDRESS']
        match_city = best_match_row.loc[0, 'CITY']
        matched_domain = best_match_row.loc[0, 'COMPANY_DOMAIN']
        #print(f"Best match: {best_match_name} with National ID: {national_id}")
        matched_ids.append(national_id)
        street.append(match_street)
        city.append(match_city)
        domain.append(matched_domain)
        names.append(best_match_name)
        match_count += 1  # Increment match count
    else:
        #print("No match found.")
        matched_ids.append(None)
        names.append(None)
        street.append(None)
        city.append(None)
        domain.append(None)

# Add the matched National IDs to nationaldataid_missing data_frame
nationaldataid_missing['NATIONAL DATA ID'] = matched_ids
nationaldataid_missing['matched_name'] = names
nationaldataid_missing['STREET_ADDRESS'] = street
nationaldataid_missing['CITY'] = city
nationaldataid_missing['COMPANY_DOMAIN'] = domain

print(f"Total number of matches: {match_count}")


Total number of matches: 1476


C:\Users\zalis\AppData\Local\Temp\ipykernel_15152\1396188897.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nationaldataid_missing['NATIONAL DATA ID'] = matched_ids
C:\Users\zalis\AppData\Local\Temp\ipykernel_15152\1396188897.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nationaldataid_missing['matched_name'] = names
C:\Users\zalis\AppData\Local\Temp\ipykernel_15152\1396188897.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [27]:
# Filter rows by district name and sort by match score to get the best match
data_nces['Match_Score'] = data_nces['DISTRICT_NAME'].apply(lambda x: fuzz.WRatio('TRACY UNIFIED SCHOOL DISTRICT', str(x)))
best_match_row = data_nces[(data_nces['STATENAME'] == state)].sort_values('Match_Score', ascending=False).reset_index()

In [28]:
# Check how does the funcrion works
best_match_row.loc[0,'DISTRICT_NAME']

'STOCKTON COLLEGIATE INTERNATIONAL SECONDARY DISTRICT'

In [30]:
# There's 3 states that had commas in their statename and were incorrectly matched
# Keep them for manual correction in the final csv.file before doing import sorted and organised data into the Strawbees Hubspot CRM DataBase.

nationaldataid_missing[nationaldataid_missing['NATIONAL DATA ID'].isna()]

Agency Total spend (approx)  \
463  Central Union High School District, Imperial C...                  NDA   
502  Greenfield Union School District, Monterey Cou...                  NDA   
524  Mountain View School District, San Bernardino ...                  NDA   

                          DISTRICT_NAME                          STATENAME  \
463  CENTRAL UNION HIGH SCHOOL DISTRICT        IMPERIAL COUNTY, CALIFORNIA   
502    GREENFIELD UNION SCHOOL DISTRICT        MONTEREY COUNTY, CALIFORNIA   
524       MOUNTAIN VIEW SCHOOL DISTRICT  SAN BERNARDINO COUNTY, CALIFORNIA   

    NATIONAL DATA ID STREET_ADDRESS  CITY COMPANY_DOMAIN matched_name  
463             None           None  None           None         None  
502             None           None  None           None         None  
524             None           None  None           None         None

In [31]:
# Check the number of all metched districts
print(f"Total number of matches: {match_count}")

Total number of matches: 1476


# Joining data frames nationaldataid_missing and nationaldataid_present

Part 5: Merging resulting dataframes for final version of school districts with all needed data ready for the import in Strawbees Hubspot CRM Database.

In [32]:
# Preparing final dataset for the import micro:bit data:
# 1. Combining both datasets
# 2. Adding "use_microbit" column with value "Yes"
# 3. Exporting to spreadsheet for manual correction

In [43]:
# Delete no needed columns
nationaldataid_missing.drop(labels=["matched_name"], axis=1, inplace=True)
nationaldataid_missing.head(2)

C:\Users\zalis\AppData\Local\Temp\ipykernel_15152\3650164116.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nationaldataid_missing.drop(labels=["matched_name"], axis=1, inplace=True)


Agency Total spend (approx)  \
0  State College Area School District, Pennsylvania                  NDA   
1               Carson City School District, Nevada                  NDA   

                        DISTRICT_NAME     STATENAME       NATIONAL DATA ID  \
0  STATE COLLEGE AREA SCHOOL DISTRICT  PENNSYLVANIA  NCES_District-4222770   
1         CARSON CITY SCHOOL DISTRICT        NEVADA  NCES_District-3200390   

       STREET_ADDRESS           CITY                COMPANY_DOMAIN  
0  240 Villa Crest Dr  State College          http://www.scasd.org  
1          PO Box 603    Carson City  http://carsoncityschools.com

In [34]:
# Show total number of values after fuzzy matching
nationaldataid_missing.shape[0]

1479

In [35]:
nationaldataid_present.head()

Agency Total spend (approx)  \
7                    Modesto City Schools, California                  NDA   
8   Stanislaus County Office of Education, California                  NDA   
10                             San Antonio ISD, Texas                  NDA   
11         Montgomery County Public Schools, Maryland                  NDA   
23            Cabarrus County Schools, North Carolina                  NDA   

                            DISTRICT_NAME       STATENAME  \
7                    MODESTO CITY SCHOOLS      CALIFORNIA   
8   STANISLAUS COUNTY OFFICE OF EDUCATION      CALIFORNIA   
10                        SAN ANTONIO ISD           TEXAS   
11       MONTGOMERY COUNTY PUBLIC SCHOOLS        MARYLAND   
23                CABARRUS COUNTY SCHOOLS  NORTH CAROLINA   

         NATIONAL DATA ID        STREET_ADDRESS         CITY  \
7   NCES_District-0601330        426 Locust St.      Modesto   
8   NCES_District-0691041            1100 H St.      Modesto   
10  NCES_District-4838730         514 QUINCY ST  SAN ANTONIO   
11  NCES_District-2400480  850 Hungerford Drive    Rockville   
23  NCES_District-3700530            PO Box 388      Concord   

                        COMPANY_DOMAIN  
7           http://www.monet.k12.ca.us  
8               http://www.stancoe.org  
10               http://www.saisd.net/  
11  http://www.montgomeryschoolsmd.org  
23      http://www.cabarrus.k12.nc.us/

In [36]:
# Show total number of values after merging original dataframes
nationaldataid_present.shape[0]

418

In [37]:
# Combining both datasets after data exploring, manipulation and merging
microbitdata_for_import = pd.concat([nationaldataid_missing, nationaldataid_present], ignore_index=True)
microbitdata_for_import.head()


Agency Total spend (approx)  \
0  State College Area School District, Pennsylvania                  NDA   
1               Carson City School District, Nevada                  NDA   
2           Gwinnett County Public Schools, Georgia                  NDA   
3         Tracy Unified School District, California                  NDA   
4                     Pasco County Schools, Florida                  NDA   

                        DISTRICT_NAME     STATENAME       NATIONAL DATA ID  \
0  STATE COLLEGE AREA SCHOOL DISTRICT  PENNSYLVANIA  NCES_District-4222770   
1         CARSON CITY SCHOOL DISTRICT        NEVADA  NCES_District-3200390   
2      GWINNETT COUNTY PUBLIC SCHOOLS       GEORGIA  NCES_District-1302550   
3       TRACY UNIFIED SCHOOL DISTRICT    CALIFORNIA  NCES_District-0601792   
4                PASCO COUNTY SCHOOLS       FLORIDA  NCES_District-1201530   

           STREET_ADDRESS           CITY                     COMPANY_DOMAIN  \
0      240 Villa Crest Dr  State College               http://www.scasd.org   
1              PO Box 603    Carson City       http://carsoncityschools.com   
2          52 Gwinnett Dr  Lawrenceville                                NaN   
3             PO Box 2286       Stockton  http://www.stocktoncollegiate.org   
4  7227 LAND O LAKES BLVD   LAND O LAKES         http://WWW.PASCO.K12.FL.US   

                                        matched_name  
0                              STATE COLLEGE AREA SD  
1                                        CARSON CITY  
2                                    GWINNETT COUNTY  
3  STOCKTON COLLEGIATE INTERNATIONAL SECONDARY DI...  
4                                              PASCO

In [38]:
# Show total number of districts 
total_districts = len(microbitdata_for_import)
print(f"Number of total districts in the merged dataframe: {total_districts}")

Number of total districts in the merged dataframe: 1897


Adding coulmn "use_microbit" (similar with the Property name in Hubspot SRM Strawbees DataBase).

In [39]:
# Adding "use_microbit" column with value "Yes" 
microbitdata_for_import["use_microbit"] = "Yes"
microbitdata_for_import.head()

Agency Total spend (approx)  \
0  State College Area School District, Pennsylvania                  NDA   
1               Carson City School District, Nevada                  NDA   
2           Gwinnett County Public Schools, Georgia                  NDA   
3         Tracy Unified School District, California                  NDA   
4                     Pasco County Schools, Florida                  NDA   

                        DISTRICT_NAME     STATENAME       NATIONAL DATA ID  \
0  STATE COLLEGE AREA SCHOOL DISTRICT  PENNSYLVANIA  NCES_District-4222770   
1         CARSON CITY SCHOOL DISTRICT        NEVADA  NCES_District-3200390   
2      GWINNETT COUNTY PUBLIC SCHOOLS       GEORGIA  NCES_District-1302550   
3       TRACY UNIFIED SCHOOL DISTRICT    CALIFORNIA  NCES_District-0601792   
4                PASCO COUNTY SCHOOLS       FLORIDA  NCES_District-1201530   

           STREET_ADDRESS           CITY                     COMPANY_DOMAIN  \
0      240 Villa Crest Dr  State College               http://www.scasd.org   
1              PO Box 603    Carson City       http://carsoncityschools.com   
2          52 Gwinnett Dr  Lawrenceville                                NaN   
3             PO Box 2286       Stockton  http://www.stocktoncollegiate.org   
4  7227 LAND O LAKES BLVD   LAND O LAKES         http://WWW.PASCO.K12.FL.US   

                                        matched_name use_microbit  
0                              STATE COLLEGE AREA SD          Yes  
1                                        CARSON CITY          Yes  
2                                    GWINNETT COUNTY          Yes  
3  STOCKTON COLLEGIATE INTERNATIONAL SECONDARY DI...          Yes  
4                                              PASCO          Yes

In [40]:
# Exporting final resulting dataframe to spreadsheet for manual correction
microbitdata_for_import.to_csv('microbitdata_for_import.csv', index=False)
